In [50]:
import random

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

In [ ]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

predicted_text

In [184]:
class TextGenerator:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # Load pre-trained model (weights)
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')

        # Set the model in evaluation mode to deactivate the DropOut modules
        # This is IMPORTANT to have reproducible results during evaluation!
        self.model.eval()
        # If you have a GPU, put everything on cuda
        self.model.to('cuda')

    
    def generate_word(self, start_text):
        """
        Generate one word (or sub-word, sometimes) to add onto some start text.
        The generated word will contain a leader space if appropriate.
        """
        # Encode text inputs
        indexed_tokens = self.tokenizer.encode(start_text)
        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])
        # Move the tokens to the GPU.
        tokens_tensor = tokens_tensor.to('cuda')
        

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word.
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        generated_word = self.tokenizer.decode(predicted_index)
        return generated_word
    
    def generate(self, start_text, word_count=7):
        text = start_text
        for _ in range(word_count):
            text += self.generate_word(text)
        return text
    
    def generate_sentence(self, start_text, sentence_count=1, up_to_count = None):
        if up_to_count:
            sentence_count = random.randint(1, up_to_count)
        sentence_count=2 # TEMPORARY
        text = start_text
        sentence = ''
        for _ in range(sentence_count):
            while (True):
                word = self.generate_word(text)
                text += word
                sentence += word
                if '.' in word:
                    break
        return sentence

gen = TextGenerator()

In [185]:
gen.generate_word("Can't stop now; I've travelled so")

' far'

In [186]:
gen.generate("Can't stop now; I've travelled",  10)

"Can't stop now; I've travelled a lot. I've been to a lot of"

In [188]:
text = gen.generate("My assignment was to wash the dishes.  I couldn't do that because",20)
text

"My assignment was to wash the dishes.  I couldn't do that because I was too busy.               "

In [189]:
text += "I feel terrible about that because"
gen.generate(text)

"My assignment was to wash the dishes.  I couldn't do that because I was too busy.               I feel terrible about that because I didn't do it. "

In [190]:
class BlameModeMe:
    my = 'my'
    i = 'I'
    me = 'me'

class BlameModeYou:
    my = 'your'
    i = 'you'
    me = 'you'

class BlameModeTeam:
    my = 'our'
    i = 'we'
    me = 'us'

class BlameModeTeamBlameShift:
    my = 'their'
    i = 'they'
    me = 'them'


In [191]:
INTRO_TEXT = [
    '[my] assignment was to',
    '[I] was supposed to',
    '[I] intended to',
    '[my] goal was to',
    '[my] dream, [my] destiny was to',
    '[I] had every intention to'
]

HOWEVER_TEXT = [
    "Sadly, that didn't work out because",
    "Unfortunately, there was a serious problem with that",
    "[I] couldn't do that because",
]

TASK_INTRO = [
    'Things were going pretty well until [I] got to the part where [I] needed to',
    '[I] hit a serious problem when [I] started to',
    '[I] hit a major roadblock when [I] began to'
]

TASK_TRANSITION = [
    'The problem was',
    'What stopped [me] dead'
]

In [192]:
random.choice(HOWEVER_TEXT)

"[I] couldn't do that because"

In [193]:
'You know that [I] would never do that'.replace('[I]', 'I')

'You know that I would never do that'

In [199]:
class ExcuseSituation:
    def __init__(self, text_generator, assignment, tasks=[], is_team=False, blame_others=False):
        self.assignment = assignment
        self.tasks = tasks
        self.generator = text_generator
        if is_team:
            if blame_others:
                mode = BlameModeTeamBlameShift
            else:
                mode = BlameModeTeam
        else:
            if blame_others:
                mode = BlameModeYou
            else:
                mode = BlameModeMe
        self.mode = mode
    
    def generate_excuse(self):
        if random.random() < 0.5 or not self.tasks:
            return self.generate_excuse_whole()
        else:
            return self.generate_excuse_task()
        
    def generate_excuse_task(self):
        background = [random.choice(INTRO_TEXT), self.assignment, '.']
        background += [random.choice(TASK_INTRO), random.choice(self.tasks), '.']
        background += [random.choice(TASK_TRANSITION)]
        background_text = self._list_to_text(background)
        text = self.generator.generate_sentence(background_text, up_to_count=3)
        return self._prep_result(background_text + text)
        
    def generate_excuse_whole(self):
        # Lead-in text, setting up the situation.
        background = [random.choice(INTRO_TEXT), self.assignment, '.']
        background += [random.choice(HOWEVER_TEXT)]
        
        background_text = self._list_to_text(background)
        text = self.generator.generate_sentence(background_text)
        return self._prep_result(background_text + text)

    def generate_excuses(self, count=5):
        result = []
        for _ in range(count):
            result.append(self.generate_excuse())
        return result
            
    def _list_to_text(self, chunk_list):
        words = []
        for entry in chunk_list:
            entry = entry.replace('[I]', self.mode.i)
            entry = entry.replace('[my]', self.mode.my)
            entry = entry.replace('[me]', self.mode.me)
            words.append(entry)
        return ' '.join(words)
    
    def _prep_result(self, excuse_text):
        return excuse_text[0].upper() + excuse_text[1:] # Capitalize sentence
        

In [196]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses(count=1)

["I was supposed to prepare a nice dinner for you for Valentine's day . I hit a serious problem when I started to plan the menu . What stopped me dead in my tracks was the fact that I had to make a decision that I didn't want to make. I had to make a decision that I didn't want to make."]

## Fight the Repetition

[GPT2 Issue: Repetition](https://github.com/huggingface/transformers/issues/1725).  Solution is to use *temperarture*.

A new TextGenerator with a temperature parameter.

In [209]:
import torch.nn.functional as F

class TextGenerator:
    def __init__(self):
        # Load pre-trained model tokenizer (vocabulary)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # Load pre-trained model (weights)
        self.model = GPT2LMHeadModel.from_pretrained('gpt2')
        # Temperature adds randomness and unpredictability.
        self.temperature = random.choice([0.8, 0.8, 0.8, 0.9, 0.7])

        # Set the model in evaluation mode to deactivate the DropOut modules
        # This is IMPORTANT to have reproducible results during evaluation!
        self.model.eval()
        # If you have a GPU, put everything on cuda
        self.model.to('cuda')

    
    def generate_word(self, start_text):
        """
        Generate one word (or sub-word, sometimes) to add onto some start text.
        The generated word will contain a leader space if appropriate.
        """
        # Encode text inputs
        indexed_tokens = self.tokenizer.encode(start_text)
        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])
        # Move the tokens to the GPU.
        tokens_tensor = tokens_tensor.to('cuda')
        

        # Predict all tokens
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            # Use temperature to add more chaos into the selection of words.
            next_token_logits = outputs[0][:, -1, :] / self.temperature   # <--- Temperature
            next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1) # <-- Temperature

        # get the predicted next sub-word.
        generated_word = self.tokenizer.decode(next_token)
        return generated_word
    
    def generate(self, start_text, word_count=7):
        text = start_text
        for _ in range(word_count):
            text += self.generate_word(text)
        return text
    
    def generate_sentence(self, start_text, sentence_count=1, up_to_count = None):
        if up_to_count:
            sentence_count = random.randint(1, up_to_count)
        sentence_count=2 # TEMPORARY
        text = start_text
        sentence = ''
        for _ in range(sentence_count):
            while (True):
                word = self.generate_word(text)
                text += word
                sentence += word
                if '.' in word:
                    break
        return sentence

gen = TextGenerator()

In [210]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses(count=4)

["I was supposed to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because I didn't feel like eating. In fact, I think pancakes are the best dessert I've ever had.",
 "I had every intention to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because my ass was still sore and I was already too sad and hungry to eat. So I decided to come up with an excuse to drive home, but then, having just finished my first breakfast, I ran into my friend, who just happened to be the new president of the United States.",
 "My assignment was to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because I was actually planning on baking this very day. One thing I had to think about was that if you run out of time and the oven starts to bake, you might want to wait till you can bake it.",
 "I intended to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because I was home from work on

## Crazy stuff this came up with:
* My dream, my destiny was to prepare a nice dinner for you for Valentine\'s day . I hit a major roadblock when I began to plan the menu . What stopped me dead in my tracks?" What stopped me dead in my tracks?" "You know what, **I want you to live inside my house for a while**, so you can **enjoy my calm and wayward, hard-working, and true love**."
* I was supposed to prepare a nice dinner for you for Valentine's day . Unfortunately, there was a serious problem with that dinner. My girlfriend thought I was **acting out as a man** so I had to be careful, but it turned out that there was a real problem.
* My assignment was to prepare a nice dinner for you for Valentine's day . Things were going pretty well until I got to the part where I needed to go to the grocery store to buy the ingredients . The problem was, I knew **there was already so much food in the fridge**. I had to look around to find the ingredients I needed and it was only over a month later when I got home that **I found the ingredient list for Trader Joe's frozen food**.
* I had every intention to prepare a nice dinner for you for Valentine's day . I hit a major roadblock when I began to plan the menu . What stopped me dead in my tracks was my own faraway world of extremities. **How could I conclude that food would be a primary way to trigger deep emotional cycles of desire?**
* My goal was to prepare a nice dinner for you for Valentine\'s day . I hit a serious problem when I started to go to the grocery store to buy the ingredients . The problem was that I had no idea how to prepare it. **I set aside my grocery money and said to myself, "where have I seen this** and how can I prove to myself how to prepare it with my own money?" **Now buy some flowers and you can actually have a good time.**
* My dream, my destiny was to prepare a nice dinner for you for Valentine's day . Sadly, that didn't work out because **I had wasted my money and we both spent a lot**. Therefore, **I did all of my annual shopping for my daughter, and it has been a pleasure to watch her grow into a confident, confident young man.**

Let's make it a little crazier.

In [206]:
gen.temperature = 0.9

In [208]:
s = ExcuseSituation(gen, assignment="prepare a nice dinner for you for Valentine's day", tasks=[
    'plan the menu',
    'go to the grocery store to buy the ingredients',
    'cook it up',
    'plate the meal in an attractive way',
])
s.generate_excuses(count=4)

["My goal was to prepare a nice dinner for you for Valentine's day . I couldn't do that because I was pregnant on this hunt myself so I haven't even tried this article yet. I believe her 10th birthday is around 3 hour away so we have a tough time on her day.",
 "I had every intention to prepare a nice dinner for you for Valentine's day . I hit a major roadblock when I began to plan the menu . The problem was, they didn't plan it all as I had planned it. But I decided to use it as I needed.",
 "I had every intention to prepare a nice dinner for you for Valentine's day . Things were going pretty well until I got to the part where I needed to plate the meal in an attractive way . What stopped me dead in my tracks was my insecurity drinking in too many restaurants and and I almost had a bath under the table because even though I ate my meals in strange and disgusting directions by empty, dank, miserable spaces like caféilles, I knew quite well that I needed to see you again.\n\n In reality